# Overview Materi

Source: https://www.youtube.com/watch?v=LDRbO9a6XPU

Jelaskan secara singkat apa itu decision tree menurut pemahamanmu!

Pohon keputusan (decision tree) merupakan salah satu metode atau algoritma dalam klasifikasi dan prediksi. Cara kerjanya adalah dengan mengajukan pertanyaan bertipe yes/no pada setiap langkah, sehingga data akan terbagi menjadi kelompok-kelompok yang semakin seragam. Proses pemisahan ini berlanjut hingga diperoleh hasil akhir berupa keputusan atau perkiraan kelas yang paling sesuai.

# Import Data & Libraries

In [9]:
from __future__ import print_function

# label kolom
header = ["color", "diameter", "label"]

# data training
training_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 3, 'Apple'],
    ['Red', 1, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],]

# data testing
testing_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 4, 'Apple'],
    ['Red', 2, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],]

# Fungsi Dasar

In [10]:
# fungsi mencari apa saja unique value dari suatu kolom
def unique_vals(rows, col):
    return set([row[col] for row in rows])

# contoh penggunaan
unique_vals(training_data, 0)




{'Green', 'Red', 'Yellow'}

In [14]:
# fungsi Menghitung jumlah unique value dari suatu kolom
def class_counts(rows):
    counts = {}
    for row in rows:
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

# contoh penggunaan
class_counts(training_data)


{'Apple': 2, 'Grape': 2, 'Lemon': 1}

In [15]:
# fungsi pengecekan suatu value numerik atau bukan
def is_numeric(value):
    return isinstance(value, int) or isinstance(value, float)

# contoh penggunaan
print(is_numeric(7))
print(is_numeric("Red"))



True
False


In [34]:
# kelas untuk merepresentasikan pertanyaan pada decision tree
class Question:
    """A Question is used to partition a dataset.

    This class records a 'column number' (e.g., 0 for Color) and a
    'column value' (e.g., 'Red'). The match method is used to compare
    the feature value in an example to the feature value stored in the
    question.
    """

    # inisialisasi kolom dan nilai pertanyaan
    def __init__(self, column, value):
        self.column = column
        self.value = value

    # mengecek apakah contoh data sesuai dengan pertanyaan
    def match(self, example):

        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value

    # mencetak pertanyaan dalam format yang mudah dibaca
    def __repr__(self):
        ...
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (
            header[self.column], condition, str(self.value))

    # menampilkan pertanyaan dalam format string yang mudah dibaca
    def __repr__(self):

        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (
            header[self.column], condition, str(self.value))

# contoh penggunaan 1
Question(1, 3)



Is diameter >= 3?

In [17]:
# membagi dataset menjadi dua berdasarkan pertanyaan
def partition(rows, question):
    ...
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows


# contoh penggunaan
true_rows, false_rows = partition(training_data, Question(0, 'Red'))
true_rows
...

Ellipsis

**apa itu gini impurity?**
<br> gini impurity berfungsi mengukur tingkat ketidakmurnian atau ketidakteraturan pada sebuah simpul (node) dalam pohon

In [18]:
# menghitung nilai Gini Impurity untuk sebuah dataset
def gini(rows):
    ...
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity


# contoh penggunaan
no_mixing = [['Apple'],
              ['Apple']]
gini(no_mixing)
...


Ellipsis

**apa itu information gain?**
<br> information gain berfungsi mengukur seberapa efektif sebuah fitur dalam memisahkan data berdasarkan kelas-kelasnya

In [19]:
# menghitung nilai Information Gain dari pemisahan dataset
def info_gain(left, right, current_uncertainty):
    ...
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

# contoh penggunaan
current_uncertainty = gini(training_data)
true_rows, false_rows = partition(training_data, Question(0, 'Green'))
info_gain(true_rows, false_rows, current_uncertainty)
...

Ellipsis

In [37]:
# mencari pertanyaan terbaik untuk membagi dataset berdasarkan information gain tertinggi
def find_best_split(rows):
    """Find the best question to ask by iterating over every feature / value
    and calculating the information gain."""
    best_gain = 0
    best_question = None
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1

    for col in range(n_features):
        values = unique_vals(rows, col)
        for val in values:

            question = Question(col, val)

            # try splitting the dataset
            true_rows, false_rows = partition(rows, question)

            # Skip this split if it doesn't divide the dataset.
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            # Calculate the information gain from this split
            gain = info_gain(true_rows, false_rows, current_uncertainty)

            # Update the best gain if we found a better split
            if gain >= best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question


# contoh penggunaan
best_gain, best_question = find_best_split(training_data)
print(best_question)

Is diameter >= 3?


# Fungsi Decision Tree

In [26]:
# merepresentasikan node daun (leaf) pada decision tree yang berisi hasil prediksi
class Leaf:


    # inisialisasi leaf dengan menghitung jumlah kemunculan tiap kelas
    def __init__(self, rows):
        self.predictions = class_counts(rows)

In [27]:
# merepresentasikan node keputusan (decision node) yang berisi pertanyaan dan cabang
class Decision_Node:



    # inisialisasi node dengan pertanyaan, cabang benar, dan cabang salah
    def __init__(self,
                 question,
                 true_branch,
                 false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch



In [53]:
# membangun decision tree secara rekursif
def build_tree(rows):
    gain, question = find_best_split(rows)
    if gain == 0:
        return Leaf(rows)
    true_rows, false_rows = partition(rows, question)
    return Decision_Node(question, true_branch, false_branch)





In [54]:
# mencetak struktur decision tree secara rekursif dalam format teks
def print_tree(node, spacing=""):
    """Prints the tree."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        print (spacing + "Predict", node.predictions)
        return

    # Print the question at this node
    print (spacing + str(node.question))

    # Call this function on the true branch
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # Call this function on the false branch
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")


# contoh penggunaan
my_tree = build_tree(training_data)
print_tree(my_tree)

NameError: name 'true_branch' is not defined

In [48]:
# mengklasifikasikan satu baris data menggunakan decision tree
def classify(row, node):

    # base case: jika sudah mencapai leaf
    if isinstance(node, Leaf):
        return node.predictions

    # menentukan apakah mengikuti cabang true atau cabang false
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

    # dengan membandingkan nilai fitur pada baris dengan pertanyaan di node



# contoh penggunaan
classify(training_data[0], my_tree)

NameError: name 'my_tree' is not defined

In [50]:
# menampilkan prediksi pada leaf dalam format persentase
def print_leaf(counts):
    ...
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs


# contoh penggunaan
print_leaf(classify(training_data[0], my_tree))

NameError: name 'my_tree' is not defined

# Predict Using Decision Tree

In [51]:
# menguji decision tree dengan data uji dan membandingkan hasil prediksi dengan label asli
for row in testing_data:

    print ("Actual: %s. Predicted: %s" %
           (row[-1], print_leaf(classify(row, my_tree))))




NameError: name 'my_tree' is not defined